In [148]:
# Открываем файл и загружаем данные
with open(f"C:\\Users\\pavlo\\Downloads\\appointments.json", "r", encoding="utf-8") as file:
    data = json.load(file)

appointments_df = pd.DataFrame(data)
appointments_df

,client_id,client_name,duration,id,service_id,service_name,start_time
0,1,Alice,40,1,2,Pedicure,2025-03-06 09:00
1,3,Charlie,30,2,3,Haircut,2025-03-06 09:50
2,1,Alice,40,3,2,Pedicure,2025-03-06 10:30
3,2,Bob,30,4,3,Haircut,2025-03-06 11:20
4,1,Alice,40,5,2,Pedicure,2025-03-06 12:00
5,3,Charlie,40,6,2,Pedicure,2025-03-06 12:50
6,3,Charlie,40,7,2,Pedicure,2025-03-06 13:40
7,3,Charlie,35,8,1,Manicure,2025-03-06 14:30
8,2,Bob,30,9,3,Haircut,2025-03-06 15:15
9,2,Bob,30,10,3,Haircut,2025-03-06 15:55


In [8]:
from datetime import datetime, time
time_obj = datetime.strptime("16:15","%H:%M").time()
datetime.combine(datetime.today().date(), time_obj)

datetime.datetime(2025, 4, 1, 16, 15)

In [149]:
WORK_HOURS_START = 9
WORK_HOURS_END = 16
BREAK_TIME = 10
APPOINTMENTS_COUNT = 20
WORK_DAYS = ['Monday','Tuesday','Wednesday','Thirsday','Friday']
services = {'Pedicure':40,'Haircut':45,'Manicure':50}
clients = ['Alice','Bob','Charlie']

In [1]:
import plotly.graph_objects as go
import pandas as pd
import json
from datetime import timedelta, datetime
import random
import bisect

def appoinments_gen(count, clients, services, work_days, work_hour_start, work_hour_end, break_time):

    appointments = []
    res = []

    print("📌 Generating non-overlapping appointments...")

    while len(appointments) < count:
        client = random.choice(clients)
        service = random.choice(list(services.keys()))
        duration = services[service]
        day = random.choice(work_days)
        hour = random.randint(work_hour_start, work_hour_end - 1)
        minute = random.choice([0, 0, 0, 10, 20, 30, 30, 40, 50])

        start_time = datetime.strptime('1970-01-05', "%Y-%m-%d").replace(hour=hour, minute=minute, second=0)
        end_time = start_time + timedelta(minutes=duration)

        overlap = any(
            (appo['day'] == day) and (
                (start_time >= appo['start_time'] and start_time < appo['end_time'] + timedelta(minutes = break_time)) or
                (end_time > appo['start_time'] - timedelta(minutes = break_time) and end_time <= appo['end_time'])
            )
            for appo in appointments
        )

        is_late = True if end_time > datetime.strptime('1970-01-05', "%Y-%m-%d").replace(hour=16, minute=0, second=0) else False

        duplicate = any(
            (appo['day'] == day and appo['client_id'] == client and appo['service_id'] == service)
            for appo in appointments
        )

        todays_appos = sorted(
            [appo for appo in appointments if appo["day"] == day],
            key=lambda x: x["start_time"]
        )

        prev_appo = None
        next_appo = None

        if todays_appos:
            start_times = [appo["start_time"] for appo in todays_appos]

            idx = bisect.bisect_right(start_times, start_time)

            if idx > 0:
                prev_appo = todays_appos[idx - 1]

            if idx < len(todays_appos):
                next_appo = todays_appos[idx]
        
        long_waiting = False
        if prev_appo is not None:
            if prev_appo['client_id'] == client and (start_time - prev_appo['end_time']).total_seconds() > break_time * 60:
                long_waiting = True

        if next_appo is not None:
            if next_appo['client_id'] == client and (next_appo['start_time'] - end_time).total_seconds() > break_time * 60:
                long_waiting = True 

        if not overlap and not duplicate and not is_late and not long_waiting:
            appointments.append({
                'day': day,
                'start_time': start_time,
                'end_time': end_time,
                'service_id': service,
                'duration': duration,
                'client_id': client
            })

            res.append({
                'client_id': client,
                'service_id': service,
                'start_time': start_time,
                'day': day
            })

    return res, appointments


In [2]:
WORK_HOURS_START = 9
WORK_HOURS_END = 16
BREAK_TIME = 10
APPOINTMENTS_COUNT = 25
WORK_DAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
services = {'Pedicure': 35, 'Haircut': 40, 'Manicure': 45}
clients = ['Alice', 'Bob', 'Charlie']

res, appointments = appoinments_gen(APPOINTMENTS_COUNT, clients, services, WORK_DAYS, WORK_HOURS_START, WORK_HOURS_END, BREAK_TIME)
pd.DataFrame(appointments).head(3)

📌 Generating non-overlapping appointments...


,day,start_time,end_time,service_id,duration,client_id
0,Wednesday,1970-01-05 11:10:00,1970-01-05 11:55:00,Manicure,45,Alice
1,Friday,1970-01-05 11:30:00,1970-01-05 12:15:00,Manicure,45,Bob
2,Thursday,1970-01-05 14:20:00,1970-01-05 15:05:00,Manicure,45,Alice


In [51]:
df = pd.DataFrame(appointments)   
week_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
df["day_order"] = df["day"].apply(lambda x: week_order.index(x))
df = df.sort_values(by=["day_order", "start_time"]).reset_index(drop=True)
df.head(30)

,day,start_time,end_time,service_id,duration,client_id,day_order
0,Monday,1970-01-05 09:30:00,1970-01-05 10:05:00,Pedicure,35,Alice,0
1,Monday,1970-01-05 11:30:00,1970-01-05 12:05:00,Pedicure,35,Charlie,0
2,Monday,1970-01-05 12:30:00,1970-01-05 13:10:00,Haircut,40,Alice,0
3,Monday,1970-01-05 13:30:00,1970-01-05 14:10:00,Haircut,40,Charlie,0
4,Monday,1970-01-05 14:30:00,1970-01-05 15:05:00,Pedicure,35,Bob,0
5,Tuesday,1970-01-05 09:00:00,1970-01-05 09:35:00,Pedicure,35,Bob,1
6,Tuesday,1970-01-05 12:10:00,1970-01-05 12:55:00,Manicure,45,Charlie,1
7,Tuesday,1970-01-05 13:30:00,1970-01-05 14:15:00,Manicure,45,Alice,1
8,Tuesday,1970-01-05 14:30:00,1970-01-05 15:10:00,Haircut,40,Charlie,1
9,Tuesday,1970-01-05 15:20:00,1970-01-05 16:00:00,Haircut,40,Bob,1


In [3]:
date_line_map={"Monday": [3.5,4.5],"Tuesday":[2.5,3.5],"Wednesday":[1.5,2.5],"Thursday":[0.5,1.5],"Friday":[-0.5,0.5]}

In [54]:
import pandas as pd
import plotly.express as px



def shedule_plot(df, current_time):
    week_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
    
    current_day, current_time = current_time.split(" ")
    current_time = datetime.strptime(f"1970-01-05 {current_time}", "%Y-%m-%d %H:%M")
    current_time = current_time.timestamp() * 1000

    if current_day not in week_order:
        print("❌ Ошибка: день вне рабочего диапазона!")
        return
        
    date_line_map={"Monday": [3.5,4.5],"Tuesday":[2.5,3.5],"Wednesday":[1.5,2.5],"Thursday":[0.5,1.5],"Friday":[-0.5,0.5]}
    
    df["day_order"] = df["day"].apply(lambda x: week_order.index(x))
    print(df.head(25))

    fig = px.timeline(
        df,
        x_start="start_time",
        x_end="end_time",
        y="day_order",
        color="client_id",
        pattern_shape="service_id",
        title="Client Appointments Schedule",
        labels={"day": ""},
        text = "client_id",
        #category_orders={"day": week_order}
    )  
    
    col_list = [1,2,3,5]
    new_template = ""
    for col in col_list:
        new_template = new_template + df.columns[col]+":"+f"%{{customdata[{col}]}}<br>"

    fig.update_traces(
            textposition='inside',
            marker=dict(line=dict(color='black', width=1)),
            customdata=df,
            hovertemplate=new_template
             
    )
    fig.update_coloraxes(showscale=False)

    fig.add_shape(
        dict(
            type="line",
            x0=current_time,
            x1=current_time,
            y0=date_line_map[current_day][0],
            y1=date_line_map[current_day][1],
            line=dict(color="red", width=3),
        )
    )

    

    fig.add_trace(go.Scatter(
        x=[current_time],
        y=[current_day],
        mode="markers",
        marker=dict(size=12, color="red", symbol="circle"),
        name="Current Time",
        hovertemplate="<b>Current Time<br>Day:</b>%{y}<br><b>Time:</b> %{x|%H:%M}<extra></extra>"
    ))

    fig.show()

In [55]:
shedule_plot(pd.DataFrame(appointments), 'Monday 10:17')

          day          start_time            end_time service_id  duration  \
0   Wednesday 1970-01-05 11:10:00 1970-01-05 11:55:00   Manicure        45   
1      Friday 1970-01-05 11:30:00 1970-01-05 12:15:00   Manicure        45   
2    Thursday 1970-01-05 14:20:00 1970-01-05 15:05:00   Manicure        45   
3   Wednesday 1970-01-05 12:10:00 1970-01-05 12:50:00    Haircut        40   
4     Tuesday 1970-01-05 12:10:00 1970-01-05 12:55:00   Manicure        45   
5    Thursday 1970-01-05 15:20:00 1970-01-05 16:00:00    Haircut        40   
6      Monday 1970-01-05 11:30:00 1970-01-05 12:05:00   Pedicure        35   
7      Monday 1970-01-05 14:30:00 1970-01-05 15:05:00   Pedicure        35   
8   Wednesday 1970-01-05 09:00:00 1970-01-05 09:35:00   Pedicure        35   
9     Tuesday 1970-01-05 13:30:00 1970-01-05 14:15:00   Manicure        45   
10     Monday 1970-01-05 12:30:00 1970-01-05 13:10:00    Haircut        40   
11  Wednesday 1970-01-05 14:30:00 1970-01-05 15:10:00    Haircut

In [209]:
import plotly.graph_objects as go
import pandas as pd

# Создаём DataFrame с координатами и штриховкой
data = {
    "start_time": ["09:00", "09:30", "11:00", "13:15", "15:15"],
    "end_time": ["10:00", "10:30", "12:00", "14:00", "16:00"],
    "day":["Monday","Tuesday","Wednesday","Thursday","Friday"],
    "service_id":['Pedicure','Manicure','Haircut','Pedicure','Manicure'],
    "client_id": ["Alice","Bob","Alice","Charlie","Mark"]
}

day_map = {
    "Monday":5,
    "Tuesday":4,
    "Wednesday":3,
    "Thursday":2,
    "Friday":1
}

df = pd.DataFrame(data)

df["start_time_dt"] = pd.to_datetime(df["start_time"], format="%H:%M")
df["end_time_dt"] = pd.to_datetime(df["end_time"], format="%H:%M")
df["day_start"] = df["day"].apply(lambda x: day_map.get(x) - 0.46)
df["day_end"] = df["day"].apply(lambda x: day_map.get(x) + 0.46)

def shedule_plot(df, current_time, output_file="static/schedule_plot.html"):
    current_day = current_time.strftime("%A")
    current_time = datetime.strptime(current_time.strftime("%H:%M"), "%H:%M")
    
    unique_services = df["service_id"].unique()
    unique_clients = df["client_id"].unique()

    available_colors = ["blue", "purple", "green", "purple", "orange", "cyan"]
    available_patterns = ["/", "\\", "x", "-", "|", "+"]

    pattern_styles = {service: available_patterns[i % len(available_patterns)] for i, service in enumerate(unique_services)}
    color_styles = {client: available_colors[i % len(available_colors)] for i, client in enumerate(unique_clients)}

    fig = go.Figure()
    for _, row in df.iterrows():
        fig.add_trace(go.Scatter(
            x=[row["start_time_dt"], row["end_time_dt"], row["end_time_dt"], row["start_time_dt"], row["start_time_dt"]],
            y=[row["day_start"], row["day_start"], row["day_end"], row["day_end"], row["day_start"]],
            mode="lines",
            fill="toself",
            fillpattern=dict(shape=pattern_styles.get(row["service_id"], "")),
            line=dict(color=color_styles.get(row['client_id']), width=2),
            hoverinfo="skip"
        ))

        fig.add_trace(go.Scatter(
            x=[row["start_time_dt"], row["end_time_dt"]],
            y=[(row["day_start"] + row["day_end"]) / 2] * 2,
            mode="text",
            text=[f"<b>  {row['client_id']} - {row['service_id']}<br>  ({row['start_time']}-{row['end_time']})</b>"],  # Сам текст
            textposition="middle right",
            showlegend=False,
            hoverinfo="skip"
        ))

    fig.update_layout(
        title="Weekly Shedule",
        xaxis=dict(
            tickformat="%H:%M",
            title="Time",
            range=[pd.to_datetime("08:55", format="%H:%M"), pd.to_datetime("16:05", format="%H:%M")],
        ),
        yaxis=dict(
            range=[0.4, 5.6], 
            title="",
            tickvals=list(day_map.values()),
            ticktext=list(day_map.keys()),
            ),
        showlegend=False,
    )

    fig.add_trace(go.Scatter(
        x=[current_time, current_time],
        y=[day_map.get(current_day) - 0.48, day_map.get(current_day) + 0.48],
        mode="lines+markers",
        marker=dict(size=10, color="red", symbol="circle"),
        name="Current Time",
        hovertemplate="<b>Current Time<br>%{x|%H:%M}",
        hovertext="None"
    ))
    print(day_map.get(current_day))

    fig.show()
    #fig.write_html(output_file)

current_time = datetime.strptime("Monday 9:30", "%A %H:%M")
shedule_plot(df, current_time, output_file="static/schedule_plot.html")

5


In [57]:
import plotly.graph_objects as go

# Координаты прямоугольника
x1, x2 = 2, 5  # Начало и конец по X
y1, y2 = 1, 4  # Нижняя и верхняя границы по Y

# Определяем точки прямоугольника (по часовой стрелке)
rect_x = [x1, x2, x2, x1, x1]  # Замыкаем контур
rect_y = [y1, y1, y2, y2, y1]  # Замыкаем контур

# Создаём фигуру
fig = go.Figure()

# Добавляем прямоугольник
fig.add_trace(go.Scatter(
    x=rect_x, y=rect_y,
    fill="toself",  # Закрашиваем фигуру
    line=dict(color="blue"),  # Цвет границы
    name="Rectangle"
))

# Настраиваем оси
fig.update_layout(
    title="Simple Rectangle",
    xaxis=dict(range=[0, 7], title="X Axis"),
    yaxis=dict(range=[0, 5], title="Y Axis"),
    showlegend=True
)

# Отображаем график
fig.show()

In [212]:

def time_add(time_obj, minutes, sign = '+'):
    delta = timedelta(minutes=minutes)
    new_time = (datetime.combine(datetime.today(), time_obj) + (delta if sign == '+' else -delta)).time()
    return new_time

In [213]:
time_add(datetime.strptime("11:00","%H:%M").time(), 10, "-")

datetime.time(10, 50)

In [2]:
from utils import appoinments_gen
import pandas as pd

pd.DataFrame(appoinments_gen(25, ['Alice','Bob','Ann'], {'Manicure':25,'Pedicure':35,'Haircat':40}, ['Monday','Tuesday','Wednesday','Thursday','Friday'], 9, 16, 10)[1])

📌 Generating non-overlapping appointments...
0
1
2
3
4
5
6
6
7
8
9
9
10
11
11
11
12
13
14
14
15
16
16
16
16
17
17
17
17
17
18
18
18
18
19
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
20
21
21
21
21
21
21
21
22
22
23
24


,day,start_time,end_time,service_id,duration,client_id
0,Monday,10:20:00,11:00:00,Haircat,40,Ann
1,Monday,09:00:00,09:35:00,Pedicure,35,Bob
2,Friday,10:50:00,11:15:00,Manicure,25,Ann
3,Monday,14:50:00,15:25:00,Pedicure,35,Ann
4,Thursday,09:00:00,09:35:00,Pedicure,35,Alice
5,Thursday,11:50:00,12:15:00,Manicure,25,Alice
6,Tuesday,15:20:00,15:55:00,Pedicure,35,Ann
7,Friday,10:00:00,10:40:00,Haircat,40,Bob
8,Wednesday,10:00:00,10:35:00,Pedicure,35,Bob
9,Thursday,13:00:00,13:35:00,Pedicure,35,Bob


In [1]:
import pandas as pd
import datetime
data = pd.DataFrame([{'day': 'Friday', 'start_time': datetime.time(9, 50), 'end_time': datetime.time(10, 15), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Alice'}, {'day': 'Tuesday', 'start_time': datetime.time(13, 0), 'end_time': datetime.time(13, 25), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Bob'}, {'day': 'Monday', 'start_time': datetime.time(11, 0), 'end_time': datetime.time(11, 35), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Bob'}, {'day': 'Wednesday', 'start_time': datetime.time(11, 50), 'end_time': datetime.time(12, 15), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Bob'}, {'day': 'Monday', 'start_time': datetime.time(9, 10), 'end_time': datetime.time(9, 35), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Bob'}, {'day': 'Thursday', 'start_time': datetime.time(14, 10), 'end_time': datetime.time(14, 35), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Alice'}, {'day': 'Friday', 'start_time': datetime.time(11, 20), 'end_time': datetime.time(11, 55), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Alice'}, {'day': 'Monday', 'start_time': datetime.time(9, 50), 'end_time': datetime.time(10, 25), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Alice'}, {'day': 'Wednesday', 'start_time': datetime.time(10, 30), 'end_time': datetime.time(11, 5), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Bob'}, {'day': 'Thursday', 'start_time': datetime.time(10, 10), 'end_time': datetime.time(10, 45), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Alice'}, {'day': 'Thursday', 'start_time': datetime.time(9, 10), 'end_time': datetime.time(9, 35), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Bob'}, {'day': 'Tuesday', 'start_time': datetime.time(11, 50), 'end_time': datetime.time(12, 25), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Alice'}, {'day': 'Tuesday', 'start_time': datetime.time(14, 0), 'end_time': datetime.time(14, 35), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Bob'}, {'day': 'Thursday', 'start_time': datetime.time(12, 0), 'end_time': datetime.time(12, 35), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Bob'}, {'day': 'Monday', 'start_time': datetime.time(13, 50), 'end_time': datetime.time(14, 15), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Alice'}, {'day': 'Wednesday', 'start_time': datetime.time(13, 30), 'end_time': datetime.time(14, 5), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Alice'}, {'day': 'Friday', 'start_time': datetime.time(12, 20), 'end_time': datetime.time(12, 45), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Bob'}, {'day': 'Wednesday', 'start_time': datetime.time(14, 20), 'end_time': datetime.time(14, 45), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Alice'}, {'day': 'Friday', 'start_time': datetime.time(14, 30), 'end_time': datetime.time(15, 5), 'service_id': 'Pedicure', 'duration': 35, 'client_id': 'Bob'}, {'day': 'Tuesday', 'start_time': datetime.time(9, 30), 'end_time': datetime.time(9, 55), 'service_id': 'Manicure', 'duration': 25, 'client_id': 'Alice'}])
data.sort_values(['day','start_time'])

,day,start_time,end_time,service_id,duration,client_id
0,Friday,09:50:00,10:15:00,Manicure,25,Alice
6,Friday,11:20:00,11:55:00,Pedicure,35,Alice
16,Friday,12:20:00,12:45:00,Manicure,25,Bob
18,Friday,14:30:00,15:05:00,Pedicure,35,Bob
4,Monday,09:10:00,09:35:00,Manicure,25,Bob
7,Monday,09:50:00,10:25:00,Pedicure,35,Alice
2,Monday,11:00:00,11:35:00,Pedicure,35,Bob
14,Monday,13:50:00,14:15:00,Manicure,25,Alice
10,Thursday,09:10:00,09:35:00,Manicure,25,Bob
9,Thursday,10:10:00,10:45:00,Pedicure,35,Alice
